# Robot App

The robot app on the STM32 comprises two tasks:

1. Control robot velocity, balance, etc. This task is run at precise intervals (e.g. every 10ms) set by a timer interrupt. The controller also sends the status (e.g. velocity, pitch angle) to the host.

2. Communication with the host, accepting commands such as speed and direction of the robot and control parameters. This is the "default" task and runs always except when interrupted by the controller.

Both tasks share a single serial connection (UART) for communication with the host. The communication task also updates control parameters (e.g. speed) used by the controller. 

Access between the tasks is arbitrated as follows:

1. UART: when sending data to the host, the communication task (briefly) disables interrupts. This ensures message integrity. 

2. Parameters (array of floats): the communication tasks writes, the controller reads parameters. Since single float writes are "atomic", no special precautions are required for single parameter updates. Multiple parameter updates (e.g. speed of both motors) require suspending interrupts.

The format of all serial communication is binary. Commands from the host start with a single byte defining the command, optionally followed by parameters. Likewise the first byte of all messages to the host indicates its type.

In [1]:
import stm32
from serial import Serial

# fix wiring bug (feel free to delete the next two lines)
from gpiozero import Button as Pin
Pin(14, pull_up=False)

stm32.exec_no_follow("""
from pyb import UART
uart = UART(3, 9600, timeout=500)

while True:
    t = uart.readchar()
    if t < 0: continue
    print(f"cmd_handler t = {t}")
    uart.writechar(t)
""")

# fix wiring issue on my board (feel free to delete)
from gpiozero import Button as Pin
Pin(14, pull_up=False)

uart = Serial(port='/dev/ttyAMA2', baudrate=9600, 
              timeout=2, write_timeout=1, exclusive=False)

for i in range(3):
    uart.write(bytes([i]))
    print("read", uart.read(1)[0])

read 0
read 1
read 2


Works as expected, except that the output from the print statement is ignored.

To fix this, we need to simultaneously monitor the stm32 repl output and run the app, presently sending and reading bytes on `/dev/ttyAMA2`. Let's use `asyncio` to achieve light-weight concurrency.

In [2]:
from serial import Serial
import asyncio
import stm32

# run asyncio from jupyter
import nest_asyncio
nest_asyncio.apply()

# fix wiring issue
from gpiozero import Button as Pin
Pin(14, pull_up=False)

stm32_code = """
from pyb import UART
uart = UART(3, 1_000_000, timeout=500)

while True:
    t = uart.readchar()
    if t < 0: continue
    print(f"cmd_handler t = {t}")
    uart.writechar(t)
"""

async def repl(cmd, dev='/dev/ttyAMA1'):
    """Send cmd to MCU, then listen & print output."""
    stm32.exec_no_follow(cmd)
    with Serial(dev, 115200, timeout=0.5, write_timeout=2, exclusive=False) as serial:
        while True:
            if serial.in_waiting:
                data = serial.read(serial.in_waiting)
                try:
                    data = data.decode()
                    data = data.replace('\n', '\n     ')
                except:
                    pass
                print(f"MCU: {data}")
                await asyncio.sleep(0)
            else:
                await asyncio.sleep(0.5)

async def main():
    uart = Serial(port='/dev/ttyAMA2', baudrate=1_000_000, 
              timeout=2, write_timeout=1, exclusive=False)
    asyncio.create_task(repl(stm32_code))
    await asyncio.sleep(1)
    for i in range(3):
        uart.write(bytes([i]))
        print("read", uart.read(1)[0])
        # cooperative multitasking: give rt a chance to run
        await asyncio.sleep(0.1)

asyncio.run(main())

read 0
read 1
read 2
MCU: cmd_handler t = 0
     cmd_handler t = 1
     cmd_handler t = 2
     


Let's setup the code for sharing state and configuration parameters between the Raspberry PI and the STM32:

In [2]:
from serial import Serial
from struct import pack, unpack
import asyncio
import stm32

# run asyncio from jupyter
import nest_asyncio
nest_asyncio.apply()

# fix wiring issue
from gpiozero import Button as Pin
Pin(14, pull_up=False)

stm32_code = """
from array import array
from pyb import UART
from struct import pack

PARAM = array('f', [0, 1, 2])

CMD_PING = 0
CMD_GET  = 1
CMD_SET  = 2

uart = UART(3, 1_000_000, timeout=500)

smv = memoryview(PARAM)
while True:
    t = uart.readchar()
    if t < 0: continue
    if t == CMD_PING:
        uart.writechar(CMD_PING)
    elif t == CMD_GET:
        index = uart.readchar()
        uart.writechar(CMD_GET)
        uart.write(pack('f', PARAM[index]))
    elif t == CMD_SET:
        index = uart.readchar()
        uart.readinto(smv[index:index+1])
    else:
        print(f"unknown command {t}")
"""

CMD_PING = 0
CMD_GET  = 1
CMD_SET  = 2

async def repl(cmd, dev='/dev/ttyAMA1'):
    """Send cmd to MCU, then listen & print output."""
    stm32.exec_no_follow(cmd)
    with Serial(dev, 115200, timeout=0.5, write_timeout=2, exclusive=False) as serial:
        while True:
            if serial.in_waiting:
                data = serial.read(serial.in_waiting)
                try:
                    data = data.decode()
                    data.replace('\n', '\n___: ')
                except:
                    pass
                print(f"MCU: {data}")
                await asyncio.sleep(0)
            else:
                await asyncio.sleep(0.5)


async def main():
    asyncio.create_task(repl(stm32_code))
    await asyncio.sleep(1)
    uart = Serial(port='/dev/ttyAMA2', baudrate=1_000_000, 
              timeout=2, write_timeout=1, exclusive=False)
    # ping
    uart.write(bytes([CMD_PING]))
    print("ping ->", uart.read(1)[0])
    await asyncio.sleep(0.1)
    # set PARAM[2] = 3.1415
    print("set[2] = 3.1415")
    uart.write(bytes([CMD_SET, 2]))
    uart.write(pack('f', 3.1415))
    # get PARAM[2]
    uart.write(bytes([CMD_GET, 2]))
    assert uart.read(1)[0] == CMD_GET
    print("get[2] ->", unpack('f', uart.read(4))[0])
    # unknown command
    uart.write(bytes([44]))
    
stm32.hard_reset()
asyncio.run(main())

ping -> 0
set[2] = 3.1415
get[2] -> 3.1414995193481445
MCU: unknown command 44



The completed code is in `$IOT_PROJECTS/robot/code`.

In [4]:
!python $IOT_PROJECTS/robot/code/rpi/duty_control.py

MCU: start Comm @ 1000000 baud
     
fs  = 2.0
pwm = 10000.0
duty1 = 0.0
CMD_STATE = 1 -87 -1 0 0 0 706 0 pitch=-87.99
CMD_STATE = 2 -87 0 0 0 0 700 1084 pitch=-87.99
duty1 = 10.0
CMD_STATE = 3 -87 0 0 10 0 2765 1076 pitch=-87.99
CMD_STATE = 4 -87 0 0 10 0 1617 3141 pitch=-87.99
duty1 = 20.0
CMD_STATE = 5 -87 1 0 20 0 704 1993 pitch=-87.99
CMD_STATE = 6 -87 0 0 20 0 700 1080 pitch=-87.99
duty1 = 30.0
CMD_STATE = 7 -87 276 0 30 0 2507 1076 pitch=-87.99
CMD_STATE = 8 -87 311 0 30 0 2757 2883 pitch=-87.99
duty1 = 40.0
CMD_STATE = 9 -87 578 0 40 0 2540 3133 pitch=-87.99
CMD_STATE = 10 -87 608 0 40 0 2176 2916 pitch=-87.99
duty1 = 50.0
CMD_STATE = 11 -87 872 0 50 0 986 2552 pitch=-87.99
CMD_STATE = 12 -87 902 0 50 0 702 1362 pitch=-87.99
duty1 = 60.0
CMD_STATE = 13 -87 1128 0 60 0 704 1078 pitch=-87.85
CMD_STATE = 14 -87 1163 0 60 0 2529 1080 pitch=-87.85
duty1 = 70.0
CMD_STATE = 15 -87 1424 0 70 0 2532 2905 pitch=-87.85
CMD_STATE = 16 -87 1451 0 70 0 2758 2908 pitch=-87.85
duty1 = 80.0
CMD

In [13]:
!python $IOT_PROJECTS/robot/code/rpi/run.py

MCU: start Comm @ 1000000 baud
     
get(PARAM_FS) =          5
get(PARAM_FS) =          7
CMD_STATE = 1 0 0 0 0 0 697 0 pitch=0.00
CMD_STATE = 2 0 0 0 0 0 693 1075 pitch=0.00
CMD_STATE = 3 0 0 0 0 0 692 1069 pitch=0.00
CMD_STATE = 4 0 0 0 0 0 2500 1068 pitch=0.00
CMD_STATE = 5 0 0 0 0 0 2505 2876 pitch=-0.24
CMD_STATE = 6 0 0 0 0 0 697 2881 pitch=-0.20
CMD_STATE = 7 0 0 0 0 0 696 1073 pitch=-0.32
CMD_STATE = 8 0 0 0 0 0 2505 1071 pitch=-0.41
CMD_STATE = 9 0 0 0 0 0 697 2880 pitch=-0.46
done!
